In [1]:
import pandas as pd

_dir = '../../data'

df = pd.concat([
 pd.read_csv(f'{_dir}/preprocessed_train.csv'),
 pd.read_csv(f'{_dir}/train_rs.csv'),
 pd.read_csv(f'{_dir}/train_rd.csv'),
])

df = df[['data', 'label']]

df.head()

,data,label
0,지금 너 스스로를 죽여달라고 애원하는 것인가 ? 아닙니다 . 죄송합니다 . 죽을 거...,0
1,길동경찰서입니다 . 시 분 마트에 폭발물을 설치할거다 . 네 ? 똑바로 들어 한번만...,0
2,너 되게 귀여운거 알지 ? 나보다 작은 남자는 첨봤어 . 그만해 . 니들 놀리는거 ...,3
3,어이 거기 예 ? ? 너 말이야 너 . 이리 오라고 무슨 일 . 너 옷 좋아보인다 ...,1
4,저기요 혹시 날이 너무 뜨겁잖아요 ? 저희 회사에서 이 선크림 파는데 한 번 손등에...,1


In [2]:
import tensorflow as tf
from keras.callbacks import EarlyStopping
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

import seaborn as sns
import matplotlib as mpl
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

%config InlineBackend.figure_format = 'retina'
 
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager.findfont(font)

class ClassifierModel():
    def __init__(self, model_name, df):
        super(ClassifierModel, self).__init__()
        
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        
        self.model = TFAutoModelForSequenceClassification.from_pretrained(
            model_name, num_labels=4, from_pt=True)
        self.model.resize_token_embeddings(len(self.tokenizer))
        
        self.model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        
#         self.model.summary()
        
        X = df['data']
        Y = tf.keras.utils.to_categorical(df['label'], num_classes=4, dtype='float32')
        
        x_train, x_val, y_train, self.y_val = train_test_split(X, Y, test_size=0.2)
        x_train, x_test, self.y_train, self.y_test = train_test_split(x_train, y_train, test_size=0.2)
        
        self.x_train = self.tokenize(x_train)
        self.x_val = self.tokenize(x_val)
        self.x_test = self.tokenize(x_test)
        
        
    def tokenize(self, x):
        return self.tokenizer(list(x), truncation=True, padding=True)
      
    def dataset(self, x, y):
        return tf.data.Dataset.from_tensor_slices((dict(x), y))
        
    def train(self):
            
        train_dataset = self.dataset(self.x_train, self.y_train).shuffle(1000).batch(64)
        val_dataset = self.dataset(self.x_val, self.y_val).shuffle(1000).batch(64)
        
        es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='auto',  patience=1)    
        
        self.model.fit(
            train_dataset, epochs=20, batch_size=64,
            validation_data=val_dataset, callbacks=es, verbose=1)
        
    def show_report(self):
        y_pred = self.model.predict(self.x_test).argmax(axis=1)
        y_true = self.y_test.argmax(axis=1)
        
        print(f'accuracy : {accuracy_score(y_true, y_pred)}')
        print(f'macro f1 : {f1_score(y_true, y_pred, average="macro")}')
        print(f'weighted f1 : {f1_score(y_true, y_pred, average="weighted")}')
        
        conf_matrix = confusion_matrix(y_true, y_pred)
        names = ["협박", "갈취", "직장내 괴롭힘", "기타 괴롭힘"]
        conf_matrix = pd.DataFrame(conf_matrix, index=names, columns=names)
        sns.heatmap(conf_matrix, annot=True, fmt="d")
        plt.title("Confusion Matrix")
        plt.show()

In [3]:
model = ClassifierModel('klue/bert-base', df)
model.train()
# model.show_report()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/20


ResourceExhaustedError:  failed to allocate memory
	 [[node tf_bert_for_sequence_classification/bert/encoder/layer_._2/intermediate/Gelu/Erf (defined at opt/conda/lib/python3.9/site-packages/transformers/models/bert/modeling_tf_bert.py:354) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_24445]

Function call stack:
train_function
